# **Getting Experience**


**Note**: *You may initialize a small Transformer model if you wish to see it in action but it is computationally expensive and not recommended to use in this environment. You should focus on using the `RNN` for this exercise.*

The objective of this module is for you to get experience in experimenting with different model parameters and see how they influence the computation of your observables. In the previous tutorial, you used a pretrained network to compute obervables but in this case, you will train your own network and use it to compute observables.

# Knowing Our ToolBox

## Set Up the File System

Set up the file system to accommodate the imports that we need

In [1]:
import os, sys

# Calculate the path to the src directory
src_path = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'src'))
print(f"Adding {src_path} to sys.path")

# Add src directory to sys.path
sys.path.append(src_path)

# Verify that the path has been added
print(f"sys.path: {sys.path}")

Adding /Users/lere/Documents/summer_school_folders/apriquot/abriQot/src to sys.path
sys.path: ['/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/lere/Documents/summer_school_folders/apriquot/abriQot/venv/lib/python3.11/site-packages', '/Users/lere/Documents/summer_school_folders/apriquot/abriQot/src']


## Interactions

### Interaction Type
We will look at the interactions between our lattice positions. `InteractionType` provides us with the option to select one of the following

- Nearest Neigbour interactions
- Nearest Nearest Neigbour interactions
- All to all interactions

You select whichever you desire.

### van der Waals Potential

Given that 

$$V_{ij} = \frac{C_6}{r_{ij}^6} \text{ where } C_6 = \Omega R_b^6$$

we get,

$$V_{ij} = \frac{\Omega R_b^6}{r_{ij}^6}$$

You can define the interaction potential by supplying

- $\Omega$: Rabi's frequency; the default is $\Omega = 1$.

- $\R_b$: Rydberg blockade radius; the default is $R_b = 7^{\frac{1}{6}}$


All these would need to be plugged into the `InteractionsInput` class. The `get_interaction` takes to inputs to return a tuple. It takes the following


- `n`: this is the width / height of your lattice. $n = 4$ for a $4 x 4$ lattice
- `interac_input`: this should be an instance of the `InteractionsInput` mentioned earlier.

and returns 

- unique_pairs - all the possible lattice pairs ${ij}$ as chosen in your interaction type
- multipliers - their multiplier coefficients $V_{ij}$

In [2]:
from interactions import InteractionsInput, InteractionType, get_interactions

In [3]:
Omega = 0.5
rydberg_blockade = pow(7, 1/6)
inter_type = InteractionType.NN
my_input = InteractionsInput(
    interaction_type = inter_type,
    Omega = Omega,
    rydberg_blockade = rydberg_blockade
)

print(f"My input looks like: \n\t{my_input}")

unique_pairs, multipliers = get_interactions(4, my_input)

My input looks like: 
	InteractionsInput(interaction_type=<InteractionType.NN: 'nn'>, Omega=0.5, rydberg_blockade=1.3830875542684884)


In [4]:
unique_pairs.shape

(24, 2)

In [5]:
multipliers.shape

(24,)

## Variational Ansatz

Now, let us look at the provided `VMC` class. This class provides you with the following methods:

- `sample`: for sampling from the network
- `logpsi`: for computing the log of your amplitudes
- `local_energy`: for computing the local energies of the sample configurations
- `loss`: for computing the loss function which in this case is the expectation of the energy
- `train`: this implements the training loop and returns the metric (energy density) over the training epochs

In this exercise, you will only need to initialize the class and call the `train` method. Every other operation will take place as defined in the class. The `VMC` class taes the following positional arguments:

- `nsamples: int` - The batch size of samples
- `n: int` - The width/height of the lattice
- `learning_rate: float` - The mdoel weights are optimized using `Adam`. This is the learning rate
- `num_epochs: int` - Number of training steps to take
- `output_dim: int` - This will be 2 since we are dealing with spin-$\frac{1}{2}$ system
- `sequence_length: int` - This amounts to $n \otimes n$ e.g. $16$ for a $4 x 4$ lattice
- `num_hidden_units: int` - Number of hidden units in our RNN. Increasing/decreasing the number of neurons impoacts the learning capacity of the network. You may want to place around with this number.

It can take the following optional keyword argumnets. **You will have to pass them to the class in this exercise**:

- `delta: Union[int, float]` - The detuning of the laser
- `Omega: Union[int, float]` - The Rabi frequency
- `interactions: Optional[Callable]` - The `get_interactions` function
- `interactions_input: Optional[Any]` - The inputs for `get_interactions` function

In [6]:
from jax import random
from rnn_model import VMC, RNNModel
from jax import numpy as jnp

In [7]:
vmc = VMC(
    nsamples = 1000, 
    n = 4, 
    learning_rate = 0.05, 
    num_epochs = 2000,
    output_dim = 2, 
    sequence_length = 16,
    num_hidden_units = 64,
    delta = 1.0,
    Omega = Omega,
    interactions_func = get_interactions,
    interactions_input = my_input,
)

vmc

VMC(nsamples=1000, n=4, learning_rate=0.05, num_epochs=2000, output_dim=2, sequence_length=16, num_hidden_units=64, delta=1.0, Omega=0.5, interactions_func=<function get_interactions at 0x104d727a0>, interactions_input=InteractionsInput(interaction_type=<InteractionType.NN: 'nn'>, Omega=0.5, rydberg_blockade=1.3830875542684884))

In [8]:
# Prepare initial variables

dummy_nsmaples = 500
sequence_length = 16
output_dim = 2
num_hidden_units = 64
d_key = random.PRNGKey(0)

# Initialize model
model = RNNModel(
    output_dim = output_dim,
    num_hidden_units = num_hidden_units
)

# Initialize model parameters
dummy_input = jnp.zeros((dummy_nsmaples, sequence_length, output_dim))
params = model.init(random.PRNGKey(0), dummy_input)


# Run training loop
e_den = vmc.train(random.PRNGKey(123), params, model)

Training started
step 0, loss: -26.771442413330078
step 100, loss: -0.5995141267776489
step 200, loss: -1.0251247882843018
step 300, loss: -1.3343902826309204
step 400, loss: -2.302990198135376
step 500, loss: -1.979296326637268
step 600, loss: -2.340848922729492
step 700, loss: -2.4475581645965576
step 800, loss: -2.4788198471069336
step 900, loss: -2.5245602130889893


# YOUR TASK

Your assignment *:) should you choose to accpet it :)* is to train your own network and use it compute off-diagonal observables.